## Handling Missing Data with Pandas

Pandas aproveita todas as capacidades de seleção do NumPy e adiciona uma série de métodos convenientes para lidar com valores ausentes. Vamos ver um de cada vez:

In [2]:
import numpy as np
import pandas as pd

### Pandas utility functions:

Da mesma forma que o `NumPy`, o pandas também possui algumas funções de utilidade para identificar e detectar valores nulos:

In [3]:
pd.isnull(np.nan)

True

In [4]:
pd.isnull(None)

True

O oposto também existe:

In [5]:
pd.notnull(None)

False

In [6]:
pd.notnull(np.nan)

False

In [7]:
pd.notnull(3)

True

Essas funções também funcionam com `Series` e `DataFrames`:

In [8]:
pd.isnull(pd.Series([1, np.nan, 7]))

0    False
1     True
2    False
dtype: bool

In [9]:
pd.notnull(pd.Series([1, np.nan, 7]))

0     True
1    False
2     True
dtype: bool

In [10]:
pd.isnull(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


### Pandas Operations with Missing Values:

O Pandas gerencia os valores ausentes de forma mais elegante do que o NumPy. Os `NaNs` não se comportarão mais como 'vírus', e as operações simplesmente os ignorarão completamente.

In [11]:
pd.Series([1, 2, np.nan]).count()

2

In [12]:
pd.Series([1, 2, np.nan]).sum()

3.0

In [13]:
pd.Series([2, 2, np.nan]).mean()

2.0

### Filtering Missing Data:

Como vimos com o NumPy, podemos combinar a seleção booleana + `pd.isnull` para filtrar esses `NaN`s e valores nulos:

In [14]:
s = pd.Series([1, 2, 3, np.nan, np.nan, 4])

pd.notnull(s)

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [15]:
s[pd.notnull(s)]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

Tanto `notnull` quanto `isnull` são métodos de `Series` e `DataFrame`s, então podemos usá-los dessa forma:

In [16]:
s.isnull()
s.notnull()
s[s.notnull()]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

### Dropping null values

A seleção booleana + `notnull()` parece um pouco _verbose_ e repetitiva. E como dissemos antes: qualquer tarefa repetitiva provavelmente terá uma maneira melhor e mais DRY (Don't Repeat Yourself). Neste caso, podemos usar o método `dropna`:

In [17]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

In [18]:
s.dropna()

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

### Dropping null values on DataFrames

Você viu como é simples remover `NaN`s com uma Series. Mas com `DataFrame`s, haverá algumas coisas a considerar, porque você não pode remover valores únicos. Você só pode remover colunas ou linhas inteiras. Vamos começar com um `DataFrame` de exemplo:

In [19]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [20]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [21]:
df.isnull().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

O comportamento padrão do `dropna` irá remover todas as linhas nas quais qualquer valor nulo estiver presente:

In [22]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


Neste caso, estamos eliminando **linhas**. As linhas contendo valores nulos são removidas do DataFrame. Você também pode usar o parâmetro `axis` para eliminar colunas contendo valores nulos:

In [23]:
df.dropna(axis=1)  # axis='columns' also works

,Column D
0,5
1,8
2,34
3,110


Neste caso, qualquer linha ou coluna que contenha **pelo menos** um valor nulo será removida. O que pode ser, dependendo do caso, muito extremo. Você pode controlar esse comportamento com o parâmetro `how`. Pode ser '`any`' ou '`all`':

In [24]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [25]:
df.dropna(how='all') # todos os valores da linha precisam ser nulos

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [26]:
df.dropna(how='any')  # default behavior

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


Você também pode usar o parâmetro `thresh` para indicar um _limite_ (um número mínimo) de valores não nulos para a linha/coluna ser mantida:

In [27]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [28]:
df.dropna(thresh=3, axis='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


### Filling null values


Às vezes, em vez de remover os valores nulos, pode ser necessário substituí-los por algum outro valor. Isso depende muito do contexto e do conjunto de dados com o qual você está trabalhando. Às vezes, um `NaN` pode ser substituído por 0, às vezes pode ser substituído pela média da amostra e, em outras ocasiões, você pode usar o valor mais próximo.

In [29]:
s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    4.0
dtype: float64

In [30]:
s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    2.5
4    2.5
5    4.0
dtype: float64

O argumento `method` é usado para preencher valores nulos com outros valores próximos a esse valor nulo:

In [31]:
s.fillna(method='ffill')

C:\Users\welli\AppData\Local\Temp\ipykernel_1900\1029756637.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s.fillna(method='ffill')


0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    4.0
dtype: float64

In [32]:
s.fillna(method='bfill')

C:\Users\welli\AppData\Local\Temp\ipykernel_1900\1641295691.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s.fillna(method='bfill')


0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
5    4.0
dtype: float64

### Filling null values on DataFrames

O método `fillna` também funciona em DataFrames e funciona de forma semelhante. As principais diferenças são que você pode especificar o `axis` (como de costume, linhas ou colunas) a ser usado para preencher os valores (especialmente para métodos) e que você tem mais controle sobre os valores passados:

In [33]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [34]:
df.fillna(method='ffill', axis=0)

C:\Users\welli\AppData\Local\Temp\ipykernel_1900\3122255136.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=0)


,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [35]:
df.fillna(method='ffill', axis=1)

C:\Users\welli\AppData\Local\Temp\ipykernel_1900\901808322.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1)


,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0
